In [ ]:
import os
import time
import torch
import pandas as pd
import numpy as np
from torch.utils.data import DataLoader, TensorDataset
from torchinfo import summary
from sklearn.preprocessing import StandardScaler


In [ ]:
model_path = "models/nba_model.pt"
device = torch.device("cpu")
model = torch.load(model_path, map_location=device)
model.eval()
summary(model)


In [ ]:
csv_path = os.getenv("NBA_TEST_DATA", "nba_test.csv")
df = pd.read_csv(csv_path)

# Assume the label column is 'score_diff' and all others are features
X = df.drop(columns=['score_diff']).values
y = df['score_diff'].values

# Normalize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Convert to tensor
X_tensor = torch.tensor(X_scaled, dtype=torch.float32)
y_tensor = torch.tensor(y, dtype=torch.float32).unsqueeze(1)

# Create DataLoader
test_dataset = TensorDataset(X_tensor, y_tensor)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


In [ ]:
model_size = os.path.getsize(model_path)
print(f"Model Size on Disk: {model_size / 1e6:.2f} MB")


In [ ]:
total_mae = 0
total = 0

with torch.no_grad():
    for features, labels in test_loader:
        outputs = model(features)
        mae = torch.abs(outputs - labels).sum().item()
        total_mae += mae
        total += labels.size(0)

mae_result = total_mae / total
print(f"Mean Absolute Error (MAE): {mae_result:.2f}")


In [ ]:
num_trials = 100
sample = X_tensor[0].unsqueeze(0)

# Warm-up
with torch.no_grad():
    model(sample)

latencies = []
with torch.no_grad():
    for _ in range(num_trials):
        start = time.time()
        _ = model(sample)
        latencies.append(time.time() - start)


In [ ]:
print(f"Inference Latency (median): {np.percentile(latencies, 50) * 1000:.2f} ms")
print(f"Inference Latency (95th percentile): {np.percentile(latencies, 95) * 1000:.2f} ms")
print(f"Inference Latency (99th percentile): {np.percentile(latencies, 99) * 1000:.2f} ms")
print(f"Inference Throughput (single sample): {num_trials / np.sum(latencies):.2f} FPS")


In [ ]:
num_batches = 50
batch_input = X_tensor[:32]

# Warm-up
with torch.no_grad():
    model(batch_input)

batch_times = []
with torch.no_grad():
    for _ in range(num_batches):
        start = time.time()
        _ = model(batch_input)
        batch_times.append(time.time() - start)

batch_fps = (batch_input.shape[0] * num_batches) / np.sum(batch_times)
print(f"Batch Throughput: {batch_fps:.2f} FPS")


In [ ]:
print(f"Model Size on Disk: {model_size/ (1e6) :.2f} MB")
print(f"Accuracy: {accuracy:.2f}% ({correct}/{total} correct)")
print(f"Inference Latency (single sample, median): {np.percentile(latencies, 50) * 1000:.2f} ms")
print(f"Inference Latency (single sample, 95th percentile): {np.percentile(latencies, 95) * 1000:.2f} ms")
print(f"Inference Latency (single sample, 99th percentile): {np.percentile(latencies, 99) * 1000:.2f} ms")
print(f"Inference Throughput (single sample): {num_trials/np.sum(latencies):.2f} FPS")
print(f"Batch Throughput: {batch_fps:.2f} FPS")